In [4]:
# For data manipulation
import numpy as np
import pandas as pd
import scipy.signal as signal

# For visualizing the data
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# For data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

# For building the model
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics

import sys
sys.path.append('../Librerias')
import dataset as ds
#no sé porqué no funciona cuando lo tengo en la carpeta de clusters
#pero si funciona cuando esta solo en la de notebooks,,, raro.

In [5]:
#load data

fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/"
fname = "datosconsu_021023_bajos.mat"
cutoff = [8/1000, 11/1000]

set =  ds.MatFileToDataFrame(fpath, fname)
df = set.get_dataframe(cutoff)
df.tail()

,Original Signal,Signal - Mean,Filtered Signal,Hilbert Transform
599995,5.210999,-0.843661,0.008744,0.008744-0.033806j
599996,5.063692,-0.990968,0.008370,0.008370-0.038726j
599997,5.012726,-1.041933,0.007971,0.007971-0.026995j
599998,4.868050,-1.186610,0.007550,0.007550-0.033919j
599999,4.806563,-1.248097,0.007108,0.007108+0.008805j


In [6]:
amplitude_env = np.abs(df['Hilbert Transform'])
inst_phase =  np.unwrap(np.angle(df['Hilbert Transform']))
inst_freq = (np.diff(inst_phase)/(2.0*np.pi)*1000)
diff_phase =  np.diff(inst_phase)
mean_phase = np.mean(diff_phase)
diff_phase = np.insert(diff_phase, 0, 0)  

In [7]:
df['Amplitude Envelope'] = amplitude_env
df['Instantaneous Phase'] = inst_phase

#filtro al gradiente de fase

cutoff = 0.1
order = 4

b,a = signal.butter(order, cutoff, btype='low')
grad_phase = signal.filtfilt(b, a, diff_phase-mean_phase)

df['Gradient Phase'] = grad_phase

In [8]:
df.head()

,Original Signal,Signal - Mean,Filtered Signal,Hilbert Transform,Amplitude Envelope,Instantaneous Phase,Gradient Phase
0,5.326740,-0.727920,-0.071002,-0.071002-0.004745j,0.071160,-3.074865,-0.059617
1,5.335618,-0.719042,-0.060395,-0.060395-0.067391j,0.090494,-2.301505,-0.032723
2,5.460895,-0.593765,-0.049986,-0.049986-0.073204j,0.088642,-2.169909,-0.007483
3,5.491145,-0.563515,-0.039782,-0.039782-0.095259j,0.103232,-1.966397,0.014630
4,5.615436,-0.439224,-0.029789,-0.029789-0.098247j,0.102663,-1.865189,0.032492


In [24]:
start_end_defectos = [
    [25014, 25042],
    [27460, 27487],
    [35630, 35658],
    [37207, 37236],
    [48343, 48372],
    [57421, 57450],
    [61722, 61750],
    [78988, 79016],
    [82306, 82336],
    [84344, 84373],
    [97845, 97875], #10
    [99883, 99914], 
    [101083, 101111],
    [131884, 131912],
    [165590, 165619],
    [166234, 166264],
    [183081, 183109],
    [187677, 187705],
    [203502, 203530],
    [219482, 219510],
    [235509, 235537], #20
    [275603, 275633],
    [277908, 277936],
    [301722, 301753],
    [322041, 322070],
    [322422, 322450],
    [339137, 339166],
    [393906, 393935],
    [395396, 395424],
    [396716, 396746],
    [412940, 412969], #30
    [413358, 413386], 
    [430261, 430289],
    [444404, 444433],
    [453962, 453990],
    [456065, 456095],
    [456863, 456892],
    [457187, 457215],
    [457390, 457419],
    [470756, 470783],
    [474540, 474568], #40
    [502372, 502400],
    [564199, 564227]
]

### **Ventanas**

Por EDA se obtiene una posible escala para la división de la ventana de tiempo. Esta corresponde entre 10.000 y 13.500 pts por ventana.
Por lo mismo, las ventanas se harán de 10.000pts con un overlap de 1.500 pts por lado.

In [20]:
window_size = 10000
overlap = 1000

num_windows = (len(df) - window_size) // (window_size - overlap) + 1

windows = []
for i in range(num_windows):
    start = i * (window_size - overlap)
    end = start + window_size
    window = df.iloc[start:end]
    windows.append(window)

In [25]:
def find_windows(start_end_defectos, windows):
    result = []
    for start, end in start_end_defectos:
        for i, window in enumerate(windows):
            if start >= window.index[0] and end <= window.index[-1]:
                result.append(i)
                break
    return result

window_indices = find_windows(start_end_defectos, windows)


In [26]:
window_indices

[2,
 2,
 3,
 4,
 5,
 6,
 6,
 8,
 9,
 9,
 10,
 10,
 11,
 14,
 18,
 18,
 20,
 20,
 22,
 24,
 26,
 30,
 30,
 33,
 35,
 35,
 37,
 43,
 43,
 43,
 45,
 45,
 47,
 49,
 50,
 50,
 50,
 50,
 50,
 52,
 52,
 55,
 62]